# Raw Bed-Echo Power

Here, we want to extract data from all the radar images of a given survey and compile that together in a single dataframe. In this case we will use ImpDAR to load the CReSIS standard focused images one at a time. CReSIS provides a bed pick, so we will take the location and bed-echo power from that pick.

The data are publicly available at https://data.cresis.ku.edu/data/rds/2013_Antarctica_Basler.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import glob, os

# For loading matlab data files
from impdar.lib.load import load
from scipy.io import loadmat

In [ ]:
### Set up the data directories ###

# a base directory where I keep data to be loaded below
data_dir = os.getenv('DATAHOME')
# quick error check to make sure $DATAHOME is set
if data_dir is None:
    raise OSError('environment variable $DATAHOME does not exist')

cresis_siple_dir = 'airborne-radar/cresis/siple-coast/'
standard_dir = data_dir + cresis_siple_dir + 'CSARP_standard/'
layer_dir = data_dir + cresis_siple_dir + 'CSARP_layerData/'
dir_names = glob.glob(standard_dir + '20*')

In [ ]:
### Initialize the pandas dataframe with variables that we want to load ###

df = pd.DataFrame(columns=['Flight Day', 'Segment', 'Frame', 'Img Num',
                            'GPS Day','Latitude', 'Longitude',
                            'Elevation','Pitch','Roll',
                            'X', 'Y',
                            'Bed Time', 'Surface Time', 
                            'H', 'h',
                            'P_bed', 'P_surf'],dtype=float)
df["Flight Day"] = df["Flight Day"].astype(str)
df["Segment"] = df["Segment"].astype(int)
df["Frame"] = df["Frame"].astype(int)
df["Img Num"] = df["Img Num"].astype(int)

In [ ]:
### Loop through all the data files in the survey, loading data from each ###

# constants
c = 3e8              # free space velocity
epsr_ice = 3.15      # ice permittivity
c_ice = c/np.sqrt(epsr_ice)   # velocity in ice

# window for peak power (in samples)
win = 20

# for each subdirectory
for dname in dir_names:
    print(dname)
    # look for files with img_02
    fnames = glob.glob(dname+'/*img_02*.mat')
    img_num = 2
    # if there are no img_02 use img_00
    if len(fnames) == 0:
        fnames = glob.glob(dname+'/Data_2*.mat')
        img_num = 0
        
    # get day and segment from file name
    flight_day = dname.split('/')[-1][:-3]
    seg = int(dname.split('/')[-1][-2:])
    print(flight_day,seg,len(fnames))
        
    # for each file in the subdirectory
    for fname in fnames:
        # get frame from file name
        frame = int(fname.split('.')[-2][-3:])
        # try to load the file with ImpDAR
        try:
            dat = load('mcords_mat',fname)[0]
        except:
            Warning('Cannot load:',fname)
            continue
        # georeference teh file from lat/long into polar stereographic
        dat.get_projected_coords('epsg:3031')    
        
        # get the picked layer times from a separate file provided by CReSIS
        layers = loadmat(layer_dir+dname[len(standard_dir):]+'/Data'+fname[-20:])['layerData']
        btime = layers[0][1][0][0][0][0][1][0][0][0][0]
        stime = layers[0][0][0][0][0][0][1][0][0][0][0]

        # from times, get the image sample and power (if not peak power then move to peak)
        bsamp = np.zeros_like(btime)
        ssamp = np.zeros_like(stime)
        pbed = np.empty_like(btime)
        psurf = np.empty_like(stime)
        # for every trace in the image
        for trace in range(dat.tnum):
            # get the bed power from the pick sample or nearby peak power
            if np.isnan(btime[trace]):
                bsamp[trace] = np.nan
                pbed[trace] = np.nan
            else:
                b_idx = np.argmin(abs(btime[trace]-dat.travel_time*1e-6))
                bsamp[trace] = b_idx
                pbed[trace] = np.max(dat.data[b_idx-win:b_idx+win,trace])
            # get the surface power from the pick sample or nearby peak power
            if np.isnan(stime[trace]):
                ssamp[trace] = np.nan
                psurf[trace] = np.nan
            else:
                s_idx = np.argmin(abs(stime[trace]-dat.travel_time*1e-6))
                ssamp[trace] = s_idx
                psurf[trace] = np.max(dat.data[max(s_idx-win,0):s_idx+win,trace])
                
        # reload the data file for a few variables that are not loaded into ImpDAR by default
        flight_data = loadmat(fname)
        elev = np.squeeze(flight_data['Elevation'])
        pitch = np.squeeze(flight_data['Pitch'])
        roll = np.squeeze(flight_data['Roll'])
        
        # calculations
        H = c_ice/2. * (btime-stime) # ice thickness
        h = c/2. * stime             # aircraft height

        # create a temporary dataframe to fill with data from this radar image
        dfi = pd.DataFrame(np.transpose([np.zeros(dat.tnum),np.zeros(dat.tnum),np.zeros(dat.tnum),np.zeros(dat.tnum),
                                         dat.decday,dat.lat,dat.long,elev,pitch,roll,
                                         dat.x_coord,dat.y_coord,
                                         btime, stime, H, h, pbed, psurf]),
                           columns=df.columns)
        dfi["Flight Day"] = dfi["Flight Day"].astype(str)
        dfi["Segment"] = dfi["Segment"].astype(int)
        dfi["Frame"] = dfi["Frame"].astype(int)
        dfi["Img Num"] = dfi["Img Num"].astype(int)
        dfi.loc[:,'Flight Day'] = flight_day
        dfi.loc[:,'Segment'] = seg
        dfi.loc[:,'Frame'] = frame
        dfi.loc[:,'Img Num'] = img_num

        # append the temporary dataframe to the full dataframe
        df = pd.concat([df,dfi],axis=0,ignore_index=True)

In [ ]:
### Save the "raw" bed power dataframe as a csv file ###

df.to_csv('Picked_Bed_Power.csv',index=False)